Based off of https://github.com/RussTedrake/manipulation/blob/iris/iris.ipynb

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time
import numpy as np

import pydrake

from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial

from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere

from meshcat import Visualizer

In [ ]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

In [ ]:
simple_collision = False
gripper_welded = True

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)

# Add planar iiwa
iiwa_urdf = FindResourceOrThrow('drake/sandbox/planar_iiwa14_one_sphere.urdf') \
    if simple_collision else FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/"
                                                 "planar_iiwa14_spheres_dense_elbow_collision.urdf")

parser.package_map().Add("iiwa_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/iiwa_description/package.xml")))
iiwa = parser.AddModelFromFile(iiwa_urdf)
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))
q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(iiwa):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1
        
# Add WSG gripper
wsg_urdf = FindResourceOrThrow("drake/sandbox/schunk_wsg_50_welded_fingers.sdf") \
    if gripper_welded else FindResourceOrThrow("drake/manipulation/models/"
                                               "wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf")
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))
gripper = parser.AddModelFromFile(wsg_urdf)
X_7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, np.pi / 2.0), [0, 0, 0.114])
plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa),
                 plant.GetFrameByName("body", gripper), X_7G)

# Add shelves
shelves = parser.AddModelFromFile(FindResourceOrThrow("drake/sandbox/shelves.sdf"))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("shelves_body", shelves), RigidTransform([0.6,0,0.4]))

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

In [ ]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()
    
    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)

    # Add planar iiwa
    iiwa_urdf = FindResourceOrThrow('drake/sandbox/planar_iiwa14_one_sphere.urdf') \
        if simple_collision else FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/"
                                                     "planar_iiwa14_spheres_dense_elbow_collision.urdf")

    parser.package_map().Add("iiwa_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/iiwa_description/package.xml")))
    iiwa = parser.AddModelFromFile(iiwa_urdf)
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))
    q0 = [-0.2, -1.2, 1.6]
    index = 0
    for joint_index in plant.GetJointIndices(iiwa):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1

    # Add WSG gripper
    wsg_urdf = FindResourceOrThrow("drake/sandbox/schunk_wsg_50_welded_fingers.sdf") \
        if gripper_welded else FindResourceOrThrow("drake/manipulation/models/"
                                                   "wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf")
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))
    gripper = parser.AddModelFromFile(wsg_urdf)
    X_7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, np.pi / 2.0), [0, 0, 0.114])
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa),
                     plant.GetFrameByName("body", gripper), X_7G)

    # Add shelves
    shelves = parser.AddModelFromFile(FindResourceOrThrow("drake/sandbox/shelves.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("shelves_body", shelves), RigidTransform([0.6,0,0.4]))
    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())
    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    vis_diagram = builder.Build()

    simulator = Simulator(vis_diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()
    with open("/tmp/spp_shelves.html", "w") as f:
        f.write(meshcat.vis.static_html())


In [ ]:
seed_points = np.array([[0.0, -2.016, 1.975],
                        [-1, -2, 0.5],
                        [0.2, -0.8, 0.5],
                        [0.07, -1.8, -0.2],
                        [-0.1, -2, -0.3]])

# traj = PiecewisePolynomial.FirstOrderHold(np.array([0, 2]), np.array([seed_points[-2], seed_points[-1]]).T)
# visualize_trajectory(traj)

In [ ]:
iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 10
iris_options.enable_ibex = False

regions = []
for i in range(seed_points.shape[0]):
    hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Volume: %6.2f \tCenter:" % ellipse.Volume(), ellipse.center(), flush=True)
    regions.append(hpoly)

In [ ]:
# Solve path planning
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(seed_points[0,:], seed_points[3,:], regions)
spp.set_max_velocity([.4, .4, .4])
spp.set_extra_control_points_per_region(5)
print(spp.num_regions())
traj = spp.Solve()
print(traj.start_time())
print(traj.end_time())

for q in traj.control_points():
    if not any([r.PointInSet(q) for r in regions]):
        print(f"control point {q} in not in any region")
        

vis.delete()
visualize_trajectory(traj)

In [ ]:
def get_ctrl_plant():
    plant = MultibodyPlant(time_step=0.0)
    parser = Parser(plant)

    # Add planar iiwa
    iiwa_urdf = FindResourceOrThrow('drake/sandbox/planar_iiwa14_one_sphere.urdf') \
        if simple_collision else FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/"
                                                     "planar_iiwa14_spheres_dense_elbow_collision.urdf")

    parser.package_map().Add("iiwa_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/iiwa_description/package.xml")))
    iiwa = parser.AddModelFromFile(iiwa_urdf)
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))
    q0 = [-0.2, -1.2, 1.6]
    index = 0
    for joint_index in plant.GetJointIndices(iiwa):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1

    # Add WSG gripper
    wsg_urdf = FindResourceOrThrow("drake/sandbox/schunk_wsg_50_welded_fingers.sdf") \
        if gripper_welded else FindResourceOrThrow("drake/manipulation/models/"
                                                   "wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf")
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))
    gripper = parser.AddModelFromFile(wsg_urdf)
    X_7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, np.pi / 2.0), [0, 0, 0.114])
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa),
                     plant.GetFrameByName("body", gripper), X_7G)

    # Add shelves
    shelves = parser.AddModelFromFile(FindResourceOrThrow("drake/sandbox/shelves.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("shelves_body", shelves), RigidTransform([0.6,0,0.4]))
    plant.Finalize()
    
    return plant

toppra_options = CalcGridPointsOptions()
gridpoints = Toppra.CalcGridPoints(traj, toppra_options)
toppra = Toppra(traj, get_ctrl_plant(), gridpoints)
toppra.AddJointVelocityLimit([-0.4, -0.4, -0.4], [0.4, 0.4, 0.4])
toppra.AddJointAccelerationLimit([-2, -2, -2], [2, 2, 2])
s_traj = toppra.SolvePathParameterization()


q(s) & s(t) -> q(t)
print(s_traj.end_time())

In [ ]:
from pydrake.trajectories import BsplineTrajectory

bspline = BsplineTrajectory()
# print(bspline.has_derivative())
print(bspline.EvalDerivative(0, 1))
# bspline_dt = bspline.MakeDerivative(1)